# Regression of Topic Characteristics on CAR

In [4]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [5]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [6]:
%%stata
foreach stage in created end {
    import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

    * Parse date
    gen day = date(date, "YMD")
    format day %td

    * Create year variable
    gen year = year(day)

    * Create quarter variable
    gen quarter = qofd(day)
    format quarter %tq

    * Ensure index is numeric (safeguard if it came in as string)
    capture confirm numeric variable index
    if _rc {
        destring index, replace ignore("[]")
    }

    * Encode categorical variables
    encode gecko_id, gen(token)
    encode space,    gen(dao)

    egen token_year = group(token year)
    egen token_quarter = group(token quarter)

    * Define windows and specs
    local windows 2 5
    local vars    d_vote d_incentives d_govcouncil d_ecosys ///
                    d_grants d_liquidity d_treasuryops

    * Label variables
    label variable d_vote         "\$\textit{Voting}_{i,t}\$"
    label variable d_incentives   "\$\textit{Incentives}_{i,t}\$"
    label variable d_govcouncil   "\$\textit{Governance Council}_{i,t}\$"
    label variable d_ecosys       "\$\textit{Ecosystem Development}_{i,t}\$"
    label variable d_grants       "\$\textit{Grants}_{i,t}\$
    label variable d_liquidity    "\$\textit{Liquidity Management}_{i,t}\$"
    label variable d_treasuryops  "\$\textit{Treasury Operations}_{i

    eststo clear

    foreach w of local windows {
        preserve
        keep if index == `w'

        foreach v of local vars {
            reghdfe car_`w' `v', absorb(token_year token) vce(cluster token)
            eststo car`w'_`v'
        }

        restore
    }

    * Export a single big LaTeX table
    esttab                                                          ///
        car2_d_vote car2_d_incentives car2_d_govcouncil car2_d_ecosys ///
        car2_d_grants car2_d_liquidity car2_d_treasuryops             ///
        car5_d_vote car5_d_incentives car5_d_govcouncil car5_d_ecosys ///
        car5_d_grants car5_d_liquidity car5_d_treasuryops             ///
        using "$TABLES/reg_car_topic_`stage'.tex", replace            ///
        se star(* 0.10 ** 0.05 *** 0.01) label nogaps nocon           ///
        nonotes booktabs nomtitles                                    ///
        b(%9.3f) se(%9.2f)                                            ///
        mgroups("CAR[-2,2]" "CAR[-5,5]", span                         ///
            pattern(1 0 0 0 0 0 0 1 0 0 0 0 0 0)                      ///
            prefix(\multicolumn{@span}{c}{) suffix(}) )               ///
        substitute("\_" "_")                                          ///
        stats(fe_token fe_time N r2_a,                                ///
              fmt(0 0 %9.0fc %9.3f)                                   ///
              labels("Token FE" "Year FE" "Observations" "Adjusted R²"))
}


. foreach stage in created end {
  2.     import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv"
> , clear
  3. 
.     * Parse date
.     gen day = date(date, "YMD")
  4.     format day %td
  5. 
.     * Create year variable
.     gen year = year(day)
  6. 
.     * Create quarter variable
.     gen quarter = qofd(day)
  7.     format quarter %tq
  8. 
.     * Ensure index is numeric (safeguard if it came in as string)
.     capture confirm numeric variable index
  9.     if _rc {
 10.         destring index, replace ignore("[]")
 11.     }
 12. 
.     * Encode categorical variables
.     encode gecko_id, gen(token)
 13.     encode space,    gen(dao)
 14. 
.     egen token_year = group(token year)
 15.     egen token_quarter = group(token quarter)
 16. 
.     * Define windows and specs
.     local windows 2 5
 17.     local vars    d_vote d_incentives d_govcouncil d_ecosys ///
>                     d_grants d_liquidity d_treasuryops
 18. 
.     * Label variables
.     l